In [76]:
brasileirao_dates = {
    2003: ("2003-03-29", "2003-12-14"),
    2004: ("2004-04-21", "2004-12-19"),
    2005: ("2005-04-23", "2005-12-04"),
    2006: ("2006-04-15", "2006-12-03"),
    2007: ("2007-05-12", "2007-12-02"),
    2008: ("2008-05-10", "2008-12-07"),
    2009: ("2009-05-09", "2009-12-06"),
    2010: ("2010-05-08", "2010-12-05"),
    2011: ("2011-05-21", "2011-12-04"),
    2012: ("2012-05-19", "2012-12-02"),
    2013: ("2013-05-25", "2013-12-08"),
    2014: ("2014-04-19", "2014-12-07"),
    2015: ("2015-05-09", "2015-12-06"),
    2016: ("2016-05-14", "2016-12-11"),
    2017: ("2017-05-13", "2017-12-03"),
    2018: ("2018-04-14", "2018-12-02"),
    2019: ("2019-04-27", "2019-12-08"),
    2020: ("2020-08-08", "2021-02-25"),
    2021: ("2021-05-29", "2021-12-09"),
    2022: ("2022-04-09", "2022-11-13"),
    2023: ("2023-04-15", "2023-12-06"),
}

In [77]:

def calculate_outcomes(df, output_dir, season):
    import json
    import os
    from collections import defaultdict

    home_team_matchup_dict = defaultdict(lambda:defaultdict(lambda:defaultdict(lambda:0)))
    away_team_matchup_dict = defaultdict(lambda:defaultdict(lambda:defaultdict(lambda:0)))

    for i in range(len(df)):
        
        game = df.iloc[i]
            
        home = game["mandante"]
        away = game["visitante"]
        
        home_team_matchup_dict[home][away]["Win"] += 0
        away_team_matchup_dict[away][home]["Loss"] += 0
        away_team_matchup_dict[away][home]["Win"] += 0
        home_team_matchup_dict[home][away]["Loss"] += 0
        home_team_matchup_dict[home][away]["Draw"] += 0
        away_team_matchup_dict[away][home]["Draw"] += 0

        if home == game["vencedor"]:
            home_team_matchup_dict[home][away]["Win"] += 1
            away_team_matchup_dict[away][home]["Loss"] += 1
        elif away == game["vencedor"]:
            away_team_matchup_dict[away][home]["Win"] += 1
            home_team_matchup_dict[home][away]["Loss"] += 1
        else:
            home_team_matchup_dict[home][away]["Draw"] += 1
            away_team_matchup_dict[away][home]["Draw"] += 1
            
    home_path = os.path.join(os.path.join(output_dir, str(season)), "home_team_matchup.json")
    away_path = os.path.join(os.path.join(output_dir, str(season)), "away_team_matchup.json")
    
    with open(home_path, "w") as outfile: 
        json.dump(home_team_matchup_dict, outfile)
    
    with open(away_path, "w") as outfile: 
        json.dump(away_team_matchup_dict, outfile)
        
    return home_path, away_path
    

In [78]:
import pandas as pd

df = pd.read_csv('../campeonato-brasileiro-full.csv')
df['data'] = pd.to_datetime(df['data'], dayfirst=True)

for season in range(2003, 2024):

    for rollback in range(0,21):
    
        start = brasileirao_dates[season][0]
        end = brasileirao_dates[season][1]
        
        df_season = df[(df['data'] >= pd.Timestamp(start)) & (df['data'] <= pd.Timestamp(end))]

        home_path, away_path = calculate_outcomes(df_season, "C:/Users/jalfr/OneDrive/Desktop/illini/BRASA/Code/Illinois_Brasileirao_Analysis/notebooks/data", season)
        


In [79]:
from collections import defaultdict
import json
import os

for min_year in range(2003, 2024):
    for max_year in range(2003, 2024):
        
        if min_year < max_year:

            all_games_home = defaultdict(lambda:defaultdict(lambda:defaultdict(lambda:0)))
            all_games_away = defaultdict(lambda:defaultdict(lambda:defaultdict(lambda:0)))

            output_dir = "C:\\Users\\jalfr\\OneDrive\\Desktop\\illini\\BRASA\\Code\\Illinois_Brasileirao_Analysis\\notebooks\\data"

            for season in range(min_year, max_year+1):

                with open(f"C:\\Users\\jalfr\\OneDrive\\Desktop\\illini\\BRASA\\Code\\Illinois_Brasileirao_Analysis\\notebooks\\data\\{season}\\away_team_matchup.json", "r", encoding="utf-8") as f:
                    away_results = json.load(f)

                with open(f"C:\\Users\\jalfr\\OneDrive\\Desktop\\illini\\BRASA\\Code\\Illinois_Brasileirao_Analysis\\notebooks\\data\\{season}\\home_team_matchup.json", "r", encoding="utf-8") as f:
                    home_results = json.load(f)
                    
                for t1 in home_results:
                    for t2 in home_results[t1]:
                        
                        if t1 != t2:
                        
                            for ending in ["Win", "Draw", "Loss"]:
                                    
                                try:
                                    all_games_home[t1][t2][ending] += home_results[t1][t2][ending]
                                except:
                                    all_games_home[t1][t2][ending] = 0
                                    
                                try:
                                    all_games_away[t1][t2][ending] += away_results[t1][t2][ending]
                                except:
                                    all_games_away[t1][t2][ending] = 0

            all_probs_home = defaultdict(lambda:defaultdict(lambda:defaultdict(lambda:0)))
            all_probs_away = defaultdict(lambda:defaultdict(lambda:defaultdict(lambda:0)))

            for t1 in all_games_home:
                for t2 in all_games_home[t1]:
                    
                    if t1 != t2:
                        for ending in ["Win", "Draw", "Loss"]:
                                
                            try:
                                all_probs_home[t1][t2][ending] += all_games_home[t1][t2][ending]/sum(all_games_home[t1][t2].values())
                            except:
                                all_probs_home[t1][t2][ending] = 0
                                
                            try:
                                all_probs_away[t1][t2][ending] += all_games_away[t1][t2][ending]/sum(all_games_away[t1][t2].values())
                            except:
                                all_probs_away[t1][t2][ending] = 0


            home_path = os.path.join(os.path.join(output_dir, str(max_year)), f"home_team_matchup_probs_rollback_{min_year}.json")
            away_path = os.path.join(os.path.join(output_dir, str(max_year)), f"away_team_matchup_probs_rollback_{min_year}.json")

            with open(home_path, "w") as outfile: 
                json.dump(all_probs_home, outfile)

            with open(away_path, "w") as outfile: 
                json.dump(all_probs_away, outfile)


In [99]:

def weighted_random_choice(probabilities, k):
    import random
    from collections import Counter
    outcomes = list(probabilities.keys())
    weights = list(probabilities.values())
    return Counter(random.choices(outcomes, weights=weights, k=k)).most_common(1)[0][0]


def run_sim(teams, home_probs):

    team_dict = defaultdict(lambda:0)

    for t1 in teams:
        for t2 in teams:
            if t1 != t2:
                result = weighted_random_choice(home_probs[t1][t2], 10)
                if result == "Win":
                    team_dict[t1] += 3
                elif result == "Loss":
                    team_dict[t2] += 3
                else:
                    team_dict[t1] += 1
                    team_dict[t2] += 1
    
    return sorted(team_dict, key=lambda k: team_dict[k], reverse=True)

with open(f"C:\\Users\\jalfr\\OneDrive\\Desktop\\illini\\BRASA\\Code\\Illinois_Brasileirao_Analysis\\notebooks\\data\\{season}\\home_team_matchup_probs_rollback_2003.json", "r", encoding="utf-8") as f:
    probs = json.load(f)
        
teams = ["Flamengo", "Botafogo-RJ", "Fluminense", "Vasco"]

run_sim(teams, probs)

['Flamengo', 'Vasco', 'Fluminense', 'Botafogo-RJ']